# Training a model with a language model with a text dataset in fast.ai
Example of training a language model with fast.ai with a curated text dataset. 


In [1]:
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.text.all import *


In [2]:
# set up the notebook for fast.ai
fastbook.setup_book()

In [3]:
# ingest the curated text dataset ADULT_SAMPLE
path = untar_data(URLs.IMDB_SAMPLE)


In [4]:
path.ls()

(#5) [Path('/storage/data/imdb_sample/texts.csv'),Path('/storage/data/imdb_sample/imdb_sample_model.pkl'),Path('/storage/data/imdb_sample/imdb_language_model.pkl'),Path('/storage/data/imdb_sample/models'),Path('/storage/data/imdb_sample/imdb_sample.pkl')]

In [5]:
# ingest the train dataset into a Pandas dataframe
df_train = pd.read_csv(path/'texts.csv')
df_train.head(2)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False
1,positive,"This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is som...",False


In [6]:
# get the tokens and token counts for the dataframe
df_tok, count = tokenize_df(df_train,[df_train.columns[1]])

In [7]:
df_tok.head(3)

,label,is_valid,text,text_length
0,negative,False,"[xxbos, xxmaj, un, -, bleeping, -, believable, !, xxmaj, meg, xxmaj, ryan, does, n't, even, look, her, usual, pert, lovable, self, in, this, ,, which, normally, makes, me, forgive, her, shallow, ticky, acting, schtick, ., xxmaj, hard, to, believe, she, was, the, producer, on, this, dog, ., xxmaj, plus, xxmaj, kevin, xxmaj, kline, :, what, kind, of, suicide, trip, has, his, career, been, on, ?, xxmaj, whoosh, …, xxmaj, banzai, xxrep, 3, !, xxmaj, finally, this, was, directed, by, the, guy, who, did, xxmaj, big, xxmaj, chill, ?, xxmaj, must, be, a, replay, of, xxmaj, jonestown, -, hollywood,...",108
1,positive,False,"[xxbos, xxmaj, this, is, a, extremely, well, -, made, film, ., xxmaj, the, acting, ,, script, and, camera, -, work, are, all, first, -, rate, ., xxmaj, the, music, is, good, ,, too, ,, though, it, is, mostly, early, in, the, film, ,, when, things, are, still, relatively, cheery, ., xxmaj, there, are, no, really, superstars, in, the, cast, ,, though, several, faces, will, be, familiar, ., xxmaj, the, entire, cast, does, an, excellent, job, with, the, script, ., \n\n, xxmaj, but, it, is, hard, to, watch, ,, because, there, is, no, good, end, to, a, situation, like, the, one, ...]",462
2,negative,False,"[xxbos, xxmaj, every, once, in, a, long, while, a, movie, will, come, along, that, will, be, so, awful, that, i, feel, compelled, to, warn, people, ., xxmaj, if, i, labor, all, my, days, and, i, can, save, but, one, soul, from, watching, this, movie, ,, how, great, will, be, my, joy, ., \n\n, xxmaj, where, to, begin, my, discussion, of, pain, ., xxmaj, for, starters, ,, there, was, a, musical, montage, every, five, minutes, ., xxmaj, there, was, no, character, development, ., xxmaj, every, character, was, a, stereotype, ., xxmaj, we, had, swearing, guy, ,, fat, guy, who, eats, donuts, ...]",220


In [8]:
count.most_common(10)

[('xxmaj', 24930),
 ('the', 14467),
 (',', 11834),
 ('.', 11738),
 ('and', 6949),
 ('a', 6782),
 ('of', 6370),
 ('to', 5847),
 ('is', 4429),
 ('it', 4071)]

In [9]:
# get the count value for a very common word, a moderately common
# word and a rare word
print("very common word (count['the']):", count['the'])
print("moderately common word (count['prepared']):", count['prepared'])
print("rare word (count['gaga']):", count['gaga'])

very common word (count['the']): 14467
moderately common word (count['prepared']): 6
rare word (count['gaga']): 0


# Create and train the model
- create TextDataLoaders object
- define and train model

In [10]:
dls = TextDataLoaders.from_df(df_tok, path=path, 
    vocab = make_vocab(count),text_col = 'text', is_lm=True)

In [11]:
#define the model
learn = language_model_learner(dls,AWD_LSTM,metrics=accuracy)


In [12]:
# fit the model with one epoch, LR = 0.02, and momentum = ( 0.8 , 0.7 , 0.8 )
# learn.fit_one_cycle( 1 , 0.02, moms = ( 0.8 , 0.7 , 0.8 ) )
learn.fit_one_cycle( 1 , 0.02 )

epoch,train_loss,valid_loss,accuracy,time
0,1.639356,1.457713,0.771113,01:09


In [13]:
preds = learn.predict('The star is', n_words=20)

In [14]:
preds

"The star is words ' to ' , sharing that complex with typical of vargas , most of good ' grinch ' ,"

In [15]:
# .str.replace(',',''), '\''
preds2 = preds.replace(', ','').replace('\' ','')

In [16]:
preds2

'The star is words to sharing that complex with typical of vargas most of good grinch ,'

# Fine tune the model
Save the model and then fine tune it with additional training epochs

In [17]:
# save the model in /storage/data/wikitext-2/wikitext_tiny_model.pkl
learn.export('imdb_language_model.pkl')

In [18]:
learn.save("language_model_single_epoch")

Path('/storage/data/imdb_sample/models/language_model_single_epoch.pth')

In [21]:
learn2 = learn.load("language_model_single_epoch")

In [22]:
learn2.unfreeze()
learn2.fit_one_cycle(5, 0.002)

epoch,train_loss,valid_loss,accuracy,time
0,1.272196,1.158457,0.800473,01:10
1,1.181744,1.100520,0.806105,01:10
2,1.112043,1.064302,0.810819,01:11
3,1.064186,1.045866,0.812648,01:11
4,1.041773,1.042548,0.812958,01:13


In [27]:
preds = learn2.predict('The star is', n_words=20)

In [28]:
preds

"The star is ' , ' dealing '"

In [25]:
# .str.replace(',',''), '\''
preds2 = preds.replace(', ','').replace('\' ','')

In [26]:
preds2

'The star is predictable like recently found'